### Generate Optimal Trajectory Emission Files

In [ ]:
import scipy.io
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from trajectory.env.energy_models import PFM2019RAV4

energy_model = PFM2019RAV4()

base_dir = "/home/aboudy/Documents/Code/trajectory_training/dataset/data_v2_preprocessed_west"
trajectory_directories = os.listdir(base_dir)

directories = os.listdir('optimal/data/')

for directory in directories:
    print(directory)
    mat = scipy.io.loadmat('optimal/data/{}/results.mat'.format(directory))

    name = next(x for x in trajectory_directories if directory.split("_")[0] in x)

    df = pd.read_csv(os.path.join(base_dir, name, "trajectory.csv"))

    lead_speed = np.array(df["Velocity"] / 3.6)[:-200]
    lead_pos = np.array(df["DistanceGPS"] - df.at[0, "DistanceGPS"] + 92.01981072331067)[:-200]
    pos = mat['X_star'].flatten()[:-200]
    headway = lead_pos - pos - 5
    speed = mat['V_star'].flatten()[:-200]
    accel = mat['U_star'].flatten()[:-200]
    energy = [energy_model.get_instantaneous_fuel_consumption(x, y, 0) for x, y in zip(accel, speed)]

    data = {
        "time": list(0.1 * np.arange(len(headway))),
        "id": ["av"] * len(headway),
        "headway": list(headway),
        "target_accel_no_noise_no_failsafe": list(accel),
        "accel": list(accel),
        "speed": list(speed),
        "leader_speed": list(lead_speed),
        "instant_energy_consumption": energy,
    }

    pd.DataFrame(data).to_csv(os.path.join("optimal/emissions/{}.csv".format(name)), index=False)

### Plot Replay Buffer Values

In [12]:
import pickle
import os
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import numpy as np
import pandas as pd

num_agents = 1
cmap = cm.get_cmap('tab20')

filenames = ["2021-03-08-22-35-14_2T3MWRFVXLW056972_masterArray_0_4597",
             "2021-03-08-22-35-14_2T3MWRFVXLW056972_masterArray_1_4927",
             "2021-03-09-13-35-04_2T3MWRFVXLW056972_masterArray_0_6825",
             "2021-03-09-13-35-04_2T3MWRFVXLW056972_masterArray_1_4938",
             "2021-03-10-21-54-16_2T3MWRFVXLW056972_masterArray_0_4523",
             "2021-03-10-21-54-16_2T3MWRFVXLW056972_masterArray_1_4582",
             "2021-03-12-22-20-57_2T3MWRFVXLW056972_masterArray_0_5672",
             "2021-03-12-22-20-57_2T3MWRFVXLW056972_masterArray_1_4817",
             "2021-03-15-12-46-38_2T3MWRFVXLW056972_masterArray_0_4917",  # comment for optimal
             "2021-03-15-12-46-38_2T3MWRFVXLW056972_masterArray_1_11342",
             "2021-03-17-21-37-10_2T3MWRFVXLW056972_masterArray_0_4463",
             "2021-03-17-21-37-10_2T3MWRFVXLW056972_masterArray_1_4386",
             "2021-03-18-12-42-14_2T3MWRFVXLW056972_masterArray_0_3977",
             "2021-03-18-12-42-14_2T3MWRFVXLW056972_masterArray_1_3918",
             "2021-03-22-22-23-58_2T3MWRFVXLW056972_masterArray_0_4223",
             "2021-03-22-22-23-58_2T3MWRFVXLW056972_masterArray_1_4422",
             "2021-03-23-21-50-02_2T3MWRFVXLW056972_masterArray_1_3778",
             "2021-03-24-12-39-15_2T3MWRFVXLW056972_masterArray_1_4102",
             "2021-03-24-21-34-31_2T3MWRFVXLW056972_masterArray_0_4937",
             "2021-03-24-21-34-31_2T3MWRFVXLW056972_masterArray_1_4364",
             "2021-03-26-21-26-45_2T3MWRFVXLW056972_masterArray_0_4540",
             "2021-03-26-21-26-45_2T3MWRFVXLW056972_masterArray_1_4028",
             "2021-03-29-12-47-15_2T3MWRFVXLW056972_masterArray_0_5016",
             "2021-03-29-12-47-15_2T3MWRFVXLW056972_masterArray_1_4185",
             "2021-03-31-21-39-05_2T3MWRFVXLW056972_masterArray_0_4200",
             "2021-03-31-21-39-05_2T3MWRFVXLW056972_masterArray_1_4622",
             "2021-04-02-21-31-47_2T3MWRFVXLW056972_masterArray_0_4125",
             "2021-04-02-21-31-47_2T3MWRFVXLW056972_masterArray_1_4111",
             "2021-04-05-21-39-05_2T3MWRFVXLW056972_masterArray_0_4357",
             "2021-04-05-21-39-05_2T3MWRFVXLW056972_masterArray_1_4173",
             "2021-04-06-21-18-22_2T3MWRFVXLW056972_masterArray_0_4427",
             "2021-04-06-21-18-22_2T3MWRFVXLW056972_masterArray_1_4496",
             "2021-04-07-12-33-03_2T3MWRFVXLW056972_masterArray_0_11294",
             "2021-04-07-12-33-03_2T3MWRFVXLW056972_masterArray_1_6116",
             "2021-04-07-21-22-07_2T3MWRFVXLW056972_masterArray_0_4101",
             "2021-04-07-21-22-07_2T3MWRFVXLW056972_masterArray_1_4069",
             "2021-04-12-21-34-57_2T3MWRFVXLW056972_masterArray_0_4796",
             "2021-04-12-21-34-57_2T3MWRFVXLW056972_masterArray_1_4436",
             "2021-04-15-21-32-46_2T3MWRFVXLW056972_masterArray_0_3889",
             "2021-04-16-12-34-41_2T3MWRFVXLW056972_masterArray_0_5778",
             "2021-04-16-12-34-41_2T3MWRFVXLW056972_masterArray_1_4387",
             "2021-04-19-12-27-33_2T3MWRFVXLW056972_masterArray_0_16467",
             "2021-04-19-12-27-33_2T3MWRFVXLW056972_masterArray_1_6483",
             "2021-04-19-22-09-19_2T3MWRFVXLW056972_masterArray_0_4433",
             "2021-04-19-22-09-19_2T3MWRFVXLW056972_masterArray_1_4288",
             "2021-04-20-21-42-34_2T3MWRFVXLW056972_masterArray_0_4025",
             "2021-04-20-21-42-34_2T3MWRFVXLW056972_masterArray_1_3973",
             "2021-04-21-21-45-12_2T3MWRFVXLW056972_masterArray_0_3957",
             "2021-04-21-21-45-12_2T3MWRFVXLW056972_masterArray_1_3621",
             "2021-04-22-12-47-13_2T3MWRFVXLW056972_masterArray_1_5292",
             "2021-04-26-21-13-18_2T3MWRFVXLW056972_masterArray_0_4595",
             "2021-04-26-21-13-18_2T3MWRFVXLW056972_masterArray_1_4664",
             "2021-04-27-21-37-32_2T3MWRFVXLW056972_masterArray_0_3836",
             "2021-04-27-21-37-32_2T3MWRFVXLW056972_masterArray_1_3558",
             "2021-04-29-21-16-14_2T3MWRFVXLW056972_masterArray_0_4190",
             "2021-04-29-21-16-14_2T3MWRFVXLW056972_masterArray_1_4005"]

filenames = sorted([
    '2021-04-16-12-34-41_2T3MWRFVXLW056972_masterArray_0_5778',
#     '2021-03-24-12-39-15_2T3MWRFVXLW056972_masterArray_0_6438',
    '2021-03-08-22-35-14_2T3MWRFVXLW056972_masterArray_1_4927',
    '2021-03-24-21-34-31_2T3MWRFVXLW056972_masterArray_0_4937',
    '2021-04-22-12-47-13_2T3MWRFVXLW056972_masterArray_1_5292',
    '2021-03-09-13-35-04_2T3MWRFVXLW056972_masterArray_0_6825',
    '2021-03-15-12-46-38_2T3MWRFVXLW056972_masterArray_1_11342',
    '2021-04-19-12-27-33_2T3MWRFVXLW056972_masterArray_1_6483',
    '2021-03-09-13-35-04_2T3MWRFVXLW056972_masterArray_1_4938',
    '2021-03-17-21-37-10_2T3MWRFVXLW056972_masterArray_0_4463',
#     '2021-04-22-12-47-13_2T3MWRFVXLW056972_masterArray_0_7050',
    '2021-04-12-21-34-57_2T3MWRFVXLW056972_masterArray_0_4796',
    '2021-04-07-12-33-03_2T3MWRFVXLW056972_masterArray_0_11294',
    '2021-04-19-12-27-33_2T3MWRFVXLW056972_masterArray_0_16467',
    '2021-04-07-12-33-03_2T3MWRFVXLW056972_masterArray_1_6116',
    '2021-03-29-12-47-15_2T3MWRFVXLW056972_masterArray_0_5016',
    '2021-03-12-22-20-57_2T3MWRFVXLW056972_masterArray_0_5672',
    '2021-04-27-21-37-32_2T3MWRFVXLW056972_masterArray_0_3836',
    '2021-03-15-12-46-38_2T3MWRFVXLW056972_masterArray_0_4917',
])

sum_rewards = []
def main(h_high, n):
    for i in range(len(filenames)):
        directory = "mpc/th+accel/{}/n={}".format(h_high, n)
        fp = os.path.join(directory, "{}.pkl".format(filenames[i]))

        with open(fp, "rb") as f:
            data = pickle.load(f)

        actions = [x[1] for x in data]    
        rewards = [x[2] for x in data]
        states = [x[0] for x in data]    
        next_states = [x[3] for x in data]

        #print(np.array(actions), np.array(rewards), np.array(states), np.array(next_states))

        rewards = [x[2] for x in data]
        speed = [x[0][4] for x in data]
        lead_speed = [x[0][9] for x in data]
        headway = [x[0][14] for x in data]
        accel = [x[1][0] for x in data]
        num_steps = len(rewards) // num_agents

        n_obs = len(data[0][0])
        n_downstream = int((n_obs - 15) / 2)
        avg_speed = [[] for _ in range(n_downstream)]
        for x in data:
            for j in range(n_downstream):
                avg_speed[j].append(x[0][15+j])

        print(fp, sum(rewards) * 3000 / len(rewards))
        sum_rewards.append(sum(rewards) * 3000 / len(rewards))

        timesteps = np.arange(num_steps) * 0.1

        fig, ax = plt.subplots(2, 2, figsize=(12, 9))

        # ax[0, 0].set_title("Reward", fontsize=20)
        ax[0, 0].set_xlabel("time (s)", fontsize=20)
        ax[0, 0].set_ylabel("reward", fontsize=20)
        ax[0, 0].plot(timesteps, rewards[:num_steps], label="AV 0", c=cmap(0))
        if num_agents == 2:
            ax[0, 0].plot(timesteps, rewards[num_steps:], label="AV 1", c=cmap(6))
        ax[0, 0].grid(linestyle="--")
        ax[0, 0].tick_params(axis='x', labelsize=15)
        ax[0, 0].tick_params(axis='y', labelsize=15)
        ax[0, 0].legend(fontsize=15)

        # ax[0, 1].set_title("Speed", fontsize=20)
        ax[0, 1].set_xlabel("time (s)", fontsize=20)
        ax[0, 1].set_ylabel("speed (m/s)", fontsize=20)
        ax[0, 1].plot(timesteps, speed[:num_steps], label="AV 0", c=cmap(0))
        ax[0, 1].plot(timesteps, lead_speed[:num_steps], label="AV 0 (leader)", c=cmap(1))
        if num_agents == 2:
            ax[0, 1].plot(timesteps, speed[num_steps:], label="AV 1", c=cmap(6))
            ax[0, 1].plot(timesteps, lead_speed[num_steps:], label="AV 1 (leader)", c=cmap(7))
        ax[0, 1].grid(linestyle="--")
        ax[0, 1].tick_params(axis='x', labelsize=15)
        ax[0, 1].tick_params(axis='y', labelsize=15)
        ax[0, 1].legend(fontsize=15)

        # ax[1, 0].set_title("Headway", fontsize=20)
        ax[1, 0].set_xlabel("time (s)", fontsize=20)
        ax[1, 0].set_ylabel("headway (m)", fontsize=20)
        ax[1, 0].plot(timesteps, headway[:num_steps], label="AV 0", c=cmap(0))
        if num_agents == 2:
            ax[1, 0].plot(timesteps, headway[num_steps:], label="AV 1", c=cmap(6))
        ax[1, 0].grid(linestyle="--")
        ax[1, 0].tick_params(axis='x', labelsize=15)
        ax[1, 0].tick_params(axis='y', labelsize=15)
        ax[1, 0].legend(fontsize=15)

        ax[1, 1].set_title("Acceleration", fontsize=20)
        ax[1, 1].set_xlabel("time (s)", fontsize=20)
        ax[1, 1].set_ylabel("acceleration (m/^2)", fontsize=20)
        ax[1, 1].plot(timesteps, accel[:num_steps], label="AV 0", c=cmap(0))
        if num_agents == 2:
            ax[1, 1].plot(timesteps, accel[num_steps:], label="AV 1", c=cmap(6))
        ax[1, 1].grid(linestyle="--")
        ax[1, 1].tick_params(axis='x', labelsize=15)
        ax[1, 1].tick_params(axis='y', labelsize=15)
        ax[1, 1].legend(fontsize=15)

#         ax[1, 1].set_xlabel("time (s)", fontsize=20)
#         ax[1, 1].set_ylabel("average speed (m)", fontsize=20)
#         for j in range(len(avg_speed)):
#             ax[1, 1].plot(timesteps, avg_speed[j], label="{}".format(i))
#         ax[1, 1].grid(linestyle="--")
#         ax[1, 1].tick_params(axis='x', labelsize=15)
#         ax[1, 1].tick_params(axis='y', labelsize=15)
#         ax[1, 1].legend(fontsize=15)

        plt.suptitle("{}".format(filenames[i]), fontsize=20)

        fig.tight_layout()

        plt.savefig(os.path.join(directory, "{}.png".format(filenames[i])), bbox_inches="tight")
        plt.close()

    #     plt.show()


for n in range(1, 5):
    for h_high in [100]:
        main(h_high=h_high, n=n)

mpc/th+accel/100/n=1/2021-03-08-22-35-14_2T3MWRFVXLW056972_masterArray_1_4927.pkl -73.22132390290867
mpc/th+accel/100/n=1/2021-03-09-13-35-04_2T3MWRFVXLW056972_masterArray_0_6825.pkl -35.17184396300843
mpc/th+accel/100/n=1/2021-03-09-13-35-04_2T3MWRFVXLW056972_masterArray_1_4938.pkl -87.34593343228546
mpc/th+accel/100/n=1/2021-03-12-22-20-57_2T3MWRFVXLW056972_masterArray_0_5672.pkl -15.688132998644837
mpc/th+accel/100/n=1/2021-03-15-12-46-38_2T3MWRFVXLW056972_masterArray_0_4917.pkl -106.51342510885932
mpc/th+accel/100/n=1/2021-03-15-12-46-38_2T3MWRFVXLW056972_masterArray_1_11342.pkl -69.91238738682199
mpc/th+accel/100/n=1/2021-03-17-21-37-10_2T3MWRFVXLW056972_masterArray_0_4463.pkl -124.16076686802448
mpc/th+accel/100/n=1/2021-03-24-21-34-31_2T3MWRFVXLW056972_masterArray_0_4937.pkl -74.82862460000024
mpc/th+accel/100/n=1/2021-03-29-12-47-15_2T3MWRFVXLW056972_masterArray_0_5016.pkl -137.40759643812174
mpc/th+accel/100/n=1/2021-04-07-12-33-03_2T3MWRFVXLW056972_masterArray_0_11294.pkl -41

In [ ]:
np.mean(sum_rewards)

In [ ]:
import os
import numpy as np

base_dir = "/home/aboudy/Documents/Code/DDPGfD/ddpgfd/scripts/result/baseline/CompactSedan"
# base_dir = "/home/aboudy/Documents/Code/trajectory_training/data/th-2"
# base_dir = "/home/aboudy/Documents/Code/DDPGfD/ddpgfd/scripts/result/baseline/RAV4"

all_y = []
for sub_dir in sorted(os.listdir(base_dir)):
    with open(os.path.join(base_dir, sub_dir, "logs.txt"), "r") as fi:
        x = []
        for ln in fi:
            if ln.startswith("system_mpg:"):
                x.append(ln[2:])

    all_y.append(float(x[0].split(" ")[1]))


print(np.mean(all_y), np.std(all_y), max(all_y), min(all_y))
print(all_y)

base_dir = "/home/aboudy/Documents/Code/trajectory_training/data/simulate"
# base_dir = "/home/aboudy/Documents/Code/DDPGfD/ddpgfd/scripts/result/th/1/th-1-45"
# base_dir = "/home/aboudy/Documents/Code/DDPGfD/ddpgfd/scripts/result/th/2/th-2-49"
# base_dir = "/home/aboudy/Documents/Code/DDPGfD/ddpgfd/scripts/result/th/3/th-3-40"
# base_dir = "/home/aboudy/Documents/Code/DDPGfD/ddpgfd/scripts/result/th/4/th-4-40"
# base_dir = "/home/aboudy/Documents/Code/DDPGfD/ddpgfd/scripts/result/th/5/th-5-49"

all_x = []
for sub_dir in sorted(os.listdir(base_dir)):
    with open(os.path.join(base_dir, sub_dir, "logs.txt"), "r") as fi:
        x = []
        for ln in fi:
            if ln.startswith("system_mpg:"):
                x.append(ln[2:])

    all_x.append(float(x[0].split(" ")[1]))


print(np.mean(all_x), np.std(all_x), max(all_x), min(all_x))
print(all_x)

In [ ]:
np.mean(np.array(all_x) / np.array(all_y)[:len(all_x)])

In [ ]:
np.array(all_x) / np.array(all_y)[:len(all_x)]

In [ ]:
np.array(all_x)

### Parse MPC data

In [1]:
import pickle
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Load data.
with open("/home/aboudy/Downloads/expert_trajectories_step250.pkl", "rb") as f:
    x = pickle.load(f)
    
for indx, xi in enumerate(x):
    print(indx)
    lead_speeds = np.zeros(xi["observation"][:, 0].shape[0] * 10)
    headways = np.zeros(xi["observation"][:, 0].shape[0] * 10)
    speeds = np.zeros(xi["observation"][:, 0].shape[0] * 10)
    actions = np.zeros(xi["observation"][:, 0].shape[0] * 10)
    positions = np.zeros(xi["observation"][:, 0].shape[0] * 10)

    for i in range(10):
        delta = np.array(
            list(xi["observation"][1:, 0] - xi["observation"][:-1, 0]) + [0])
        speeds[i::10] = xi["observation"][:, 0] + 0.1 * i * delta

#         speeds[i::10] = xi["observation"][:, 0] + 0.1 * i * xi["action"].flatten()
        actions[i::10] = xi["action"].flatten()

        delta = np.array(
            list(xi["observation"][1:, 1] - xi["observation"][:-1, 1]) + [0])
        lead_speeds[i::10] = xi["observation"][:, 1] + 0.1 * i * delta

        delta = np.array(
            list(xi["observation"][1:, 2] - xi["observation"][:-1, 2]) + [0])
        headways[i::10] = xi["observation"][:, 2] + 0.1 * i * delta

    positions[0] = -50
    for i in range(1, len(positions)):
        positions[i] = positions[i-1] + speeds[i-1] * 0.1 + 0.5 * actions[i-1] * 0.1 ** 2

    data = {
        "time": list(0.1 * np.arange(len(headways))),
        "pos": list(positions),
        "id": ["av"] * len(headways),
        "headway": list(headways),
        "target_accel_no_noise_no_failsafe": list(actions),
        "accel": list(actions),
        "speed": list(speeds),
        "leader_speed": list(lead_speeds),
    }

    pd.DataFrame(data).to_csv(os.path.join("mpc/emissions/{}.csv".format(indx)), index=False)
    plt.figure()
    plt.plot(speeds)
    plt.show()

    plt.figure()
    plt.plot(actions)
    plt.show()

    plt.figure()
    plt.plot(positions)
    plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '/home/aboudy/Downloads/expert_trajectories_step250.pkl'

### Utility Method for demonstration generation

In [9]:
"""Generate demonstrations from an emission file."""
import sys
import argparse
import pickle
import numpy as np
import pandas as pd
import os
import json
import bisect

from trajectory.env.energy_models import PFMCompactSedan

# =========================================================================== #
#                           observation parameters                            #
# =========================================================================== #

# number of backward timesteps to add to observation
N_STEPS = 5
# headway scaling term
HEADWAY_SCALE = 200.
# speed scaling term
SPEED_SCALE = 40.

# =========================================================================== #
#                              action parameters                              #
# =========================================================================== #

# scaling term for the actions (so that values range from -1 to 1)
ACTION_SCALE = 1.

# =========================================================================== #
#                         reward function parameters                          #
# =========================================================================== #

REWARD_SCALE = 0.1
H_LOW = -1
H_HIGH = 100
TH_LOW = 1.0
TH_HIGH = -1
ENERGY_STEPS = 1
DT = 0.1
DISTANCE_SCALE = 1610


def get_leader_avg_speed(segments, avg_speed, times, pos, t, n):
    t_index = bisect.bisect(times, t) - 1
    x_index = bisect.bisect(segments, pos)
    
    return list(
        avg_speed[t_index, x_index:x_index+n] / SPEED_SCALE) + list(
        (np.array(segments[x_index:x_index+n]) - pos) / DISTANCE_SCALE)


def huber_loss(a, delta=1.0):
    if abs(a) <= delta:
        return 0.5 * a ** 2
    else:
        return delta * (abs(a) - 0.5 * delta)


def reward_fn(headway, accel, realized_accel, speed, leader_speed, avg_speed,
              segments, times, t, energy_model):
    reward = 0.

    # time headway reward
    th = min(headway[t] / max(speed[t], 0.01), 20)
    if TH_LOW > 0 and th < TH_LOW:
        reward -= 2 * huber_loss(th - TH_LOW, delta=1.00)
    if TH_HIGH > 0 and th > TH_HIGH:
        reward -= huber_loss(th - TH_HIGH, delta=0.25)

    # space headway reward
    h = headway[t]
    if H_LOW > 0 and h < H_LOW:
        reward -= 2 * huber_loss((h - H_LOW) / 10., delta=1.00)
    if H_HIGH > 0 and h > H_HIGH:
        reward -= huber_loss((h - H_HIGH) / 10., delta=0.25)

    # acceleration reward
    reward -= 0.5 * realized_accel[t] ** 2

    # failsafe reward
    # reward -= (accel[t] - realized_accel[t]) ** 2

    # energy consumption reward
#     sum_energy = sum([
#         energy_model.get_instantaneous_fuel_consumption(
#             speed=speed_i, accel=accel_i, grade=0)
#         for speed_i, accel_i in zip(speed[max(t-ENERGY_STEPS+1, 0): t+1],
#                                     accel[max(t-ENERGY_STEPS+1, 0): t+1])])
#     sum_speed = sum(np.clip(
#         speed[max(t-ENERGY_STEPS+1, 0): t+1], a_min=0.1, a_max=np.inf))

#     reward -= min(sum_energy / sum_speed, 0.2)

    return REWARD_SCALE * reward


def obs(pos, headway, accel, speed, leader_speed, avg_speed, segments, times, t, n):
    min_t = max(0, t - N_STEPS + 1)
    max_t = t + 1
    n_missed = N_STEPS - max_t + min_t

    return np.array(
        [0.] * n_missed +
        list(speed[min_t: max_t] / SPEED_SCALE) +
        [0.] * n_missed +
        list((leader_speed[min_t: max_t]-speed[min_t: max_t]) / SPEED_SCALE) +
        [0.] * n_missed +
        list(headway[min_t: max_t] / HEADWAY_SCALE) + 
        get_leader_avg_speed(segments, avg_speed, times, pos=pos[t], t=t * DT, n=n)
    )


def action(headway, accel, speed, leader_speed, avg_speed, segments, times, t):
    return np.array([ACTION_SCALE * accel[t]])


def main(infile, downstream_path, n):
    """Run the main operation."""
    # Load data.
    df = pd.read_csv(infile)

    avg_speed = None
    segments = None
    if downstream_path is not None:
        with open(os.path.join(downstream_path, "segments.json"), "r") as f:
            segments = json.load(f)

        times = sorted(list(pd.read_csv(
            os.path.join(downstream_path, "speed.csv"))["time"]))

        avg_speed = np.genfromtxt(os.path.join(
            downstream_path, "speed.csv"), delimiter=",", skip_header=1)[:, 1:]

        df_leader = pd.read_csv(os.path.join(downstream_path, "trajectory.csv"))
        expected_leader_speed = np.array(df_leader.Velocity / 3.6)

    av_ids = [x for x in np.unique(df.id) if "av" in x]
    energy_model = PFMCompactSedan()

    data = []
    for av_id in av_ids:
        # Extract data for this AV.
        df_i = df[df.id == av_id]
        df_i = df_i.sort_values("time")

        headway = np.array(df_i.headway)
        accel = np.array(df_i.target_accel_no_noise_no_failsafe)
        realized_accel = np.array(df_i.accel)
        speed = np.array(df_i.speed)
        leader_speed = np.array(df_i.leader_speed)
        pos = np.array(df_i.pos)

        for t in range(len(headway) - 1):
            s = obs(
                pos=pos,
                headway=headway,
                accel=accel,
                speed=speed,
                leader_speed=leader_speed,
                avg_speed=avg_speed,
                segments=segments,
                times=times,
                t=t,
                n=n,
            )

            s2 = obs(
                pos=pos,
                headway=headway,
                accel=accel,
                speed=speed,
                leader_speed=leader_speed,
                avg_speed=avg_speed,
                segments=segments,
                times=times,
                t=t+1,
                n=n,
            )

            a = action(
                headway=headway,
                accel=accel,
                speed=speed,
                leader_speed=leader_speed,
                avg_speed=avg_speed,
                segments=segments,
                times=times,
                t=t,
            )

            r = reward_fn(
                headway=headway,
                accel=accel,
                realized_accel=realized_accel,
                speed=speed,
                leader_speed=leader_speed,
                avg_speed=avg_speed,
                segments=segments,
                times=times,
                t=t,
                energy_model=energy_model,
            )

            # Convert data to correct format.
            data.append((s, a, r, s2))

    return data

### Random

In [29]:
INFILE=["1647581598_17Mar22_22h33m18s",
        "1647581612_17Mar22_22h33m32s",
        "1647581633_17Mar22_22h33m53s",
        "1647581656_17Mar22_22h34m16s",
        "1647581674_17Mar22_22h34m34s",
        "1647581688_17Mar22_22h34m48s",
        "1647581703_17Mar22_22h35m03s",
        "1647581726_17Mar22_22h35m26s",
        "1647581748_17Mar22_22h35m48s",
        "1647581770_17Mar22_22h36m10s",
        "1647581814_17Mar22_22h36m54s",
        "1647581830_17Mar22_22h37m10s",
        "1647581846_17Mar22_22h37m26s",
        "1647581863_17Mar22_22h37m43s",
        "1647581877_17Mar22_22h37m57s",
        "1647581891_17Mar22_22h38m11s",
        "1647581906_17Mar22_22h38m26s",
        "1647581920_17Mar22_22h38m40s",
        "1647581937_17Mar22_22h38m57s",
        "1647581956_17Mar22_22h39m16s",
        "1647581971_17Mar22_22h39m31s",
        "1647581987_17Mar22_22h39m47s",
        "1647582004_17Mar22_22h40m04s",
        "1647582023_17Mar22_22h40m23s",
        "1647582041_17Mar22_22h40m41s",
        "1647582059_17Mar22_22h40m59s",
        "1647582077_17Mar22_22h41m17s",
        "1647582093_17Mar22_22h41m33s",
        "1647582107_17Mar22_22h41m47s",
        "1647582121_17Mar22_22h42m01s",
        "1647582135_17Mar22_22h42m15s",
        "1647582151_17Mar22_22h42m31s",
        "1647582168_17Mar22_22h42m48s",
        "1647582207_17Mar22_22h43m27s",
        "1647582230_17Mar22_22h43m50s",
        "1647582244_17Mar22_22h44m04s",
        "1647582257_17Mar22_22h44m17s",
        "1647582272_17Mar22_22h44m32s",
        "1647582289_17Mar22_22h44m49s",
        "1647582307_17Mar22_22h45m07s",
        "1647582330_17Mar22_22h45m30s",
        "1647582349_17Mar22_22h45m49s",
        "1647582417_17Mar22_22h46m57s",
        "1647582440_17Mar22_22h47m20s",
        "1647582457_17Mar22_22h47m37s",
        "1647582471_17Mar22_22h47m51s",
        "1647582485_17Mar22_22h48m05s",
        "1647582498_17Mar22_22h48m18s",
        "1647582513_17Mar22_22h48m33s",
        "1647582527_17Mar22_22h48m47s",
        "1647582548_17Mar22_22h49m08s",
        "1647582565_17Mar22_22h49m25s",
        "1647582585_17Mar22_22h49m45s",
        "1647582601_17Mar22_22h50m01s",
        "1647582615_17Mar22_22h50m15s",
        "1647582631_17Mar22_22h50m31s"]

OUTFILE=["2021-03-08-22-35-14_2T3MWRFVXLW056972_masterArray_0_4597",
         "2021-03-08-22-35-14_2T3MWRFVXLW056972_masterArray_1_4927",
         "2021-03-09-13-35-04_2T3MWRFVXLW056972_masterArray_0_6825",
         "2021-03-09-13-35-04_2T3MWRFVXLW056972_masterArray_1_4938",
         "2021-03-10-21-54-16_2T3MWRFVXLW056972_masterArray_0_4523",
         "2021-03-10-21-54-16_2T3MWRFVXLW056972_masterArray_1_4582",
         "2021-03-12-22-20-57_2T3MWRFVXLW056972_masterArray_0_5672",
         "2021-03-12-22-20-57_2T3MWRFVXLW056972_masterArray_1_4817",
         "2021-03-15-12-46-38_2T3MWRFVXLW056972_masterArray_0_4917",
         "2021-03-15-12-46-38_2T3MWRFVXLW056972_masterArray_1_11342",
         "2021-03-17-21-37-10_2T3MWRFVXLW056972_masterArray_0_4463",
         "2021-03-17-21-37-10_2T3MWRFVXLW056972_masterArray_1_4386",
         "2021-03-18-12-42-14_2T3MWRFVXLW056972_masterArray_0_3977",
         "2021-03-18-12-42-14_2T3MWRFVXLW056972_masterArray_1_3918",
         "2021-03-22-22-23-58_2T3MWRFVXLW056972_masterArray_0_4223",
         "2021-03-22-22-23-58_2T3MWRFVXLW056972_masterArray_1_4422",
         "2021-03-23-21-50-02_2T3MWRFVXLW056972_masterArray_1_3778",
         "2021-03-24-12-39-15_2T3MWRFVXLW056972_masterArray_1_4102",
         "2021-03-24-21-34-31_2T3MWRFVXLW056972_masterArray_0_4937",
         "2021-03-24-21-34-31_2T3MWRFVXLW056972_masterArray_1_4364",
         "2021-03-26-21-26-45_2T3MWRFVXLW056972_masterArray_0_4540",
         "2021-03-26-21-26-45_2T3MWRFVXLW056972_masterArray_1_4028",
         "2021-03-29-12-47-15_2T3MWRFVXLW056972_masterArray_0_5016",
         "2021-03-29-12-47-15_2T3MWRFVXLW056972_masterArray_1_4185",
         "2021-03-31-21-39-05_2T3MWRFVXLW056972_masterArray_0_4200",
         "2021-03-31-21-39-05_2T3MWRFVXLW056972_masterArray_1_4622",
         "2021-04-02-21-31-47_2T3MWRFVXLW056972_masterArray_0_4125",
         "2021-04-02-21-31-47_2T3MWRFVXLW056972_masterArray_1_4111",
         "2021-04-05-21-39-05_2T3MWRFVXLW056972_masterArray_0_4357",
         "2021-04-05-21-39-05_2T3MWRFVXLW056972_masterArray_1_4173",
         "2021-04-06-21-18-22_2T3MWRFVXLW056972_masterArray_0_4427",
         "2021-04-06-21-18-22_2T3MWRFVXLW056972_masterArray_1_4496",
         "2021-04-07-12-33-03_2T3MWRFVXLW056972_masterArray_0_11294",
         "2021-04-07-12-33-03_2T3MWRFVXLW056972_masterArray_1_6116",
         "2021-04-07-21-22-07_2T3MWRFVXLW056972_masterArray_0_4101",
         "2021-04-07-21-22-07_2T3MWRFVXLW056972_masterArray_1_4069",
         "2021-04-12-21-34-57_2T3MWRFVXLW056972_masterArray_0_4796",
         "2021-04-12-21-34-57_2T3MWRFVXLW056972_masterArray_1_4436",
         "2021-04-15-21-32-46_2T3MWRFVXLW056972_masterArray_0_3889",
         "2021-04-16-12-34-41_2T3MWRFVXLW056972_masterArray_0_5778",
         "2021-04-16-12-34-41_2T3MWRFVXLW056972_masterArray_1_4387",
         "2021-04-19-12-27-33_2T3MWRFVXLW056972_masterArray_0_16467",
         "2021-04-19-12-27-33_2T3MWRFVXLW056972_masterArray_1_6483",
         "2021-04-19-22-09-19_2T3MWRFVXLW056972_masterArray_0_4433",
         "2021-04-19-22-09-19_2T3MWRFVXLW056972_masterArray_1_4288",
         "2021-04-20-21-42-34_2T3MWRFVXLW056972_masterArray_0_4025",
         "2021-04-20-21-42-34_2T3MWRFVXLW056972_masterArray_1_3973",
         "2021-04-21-21-45-12_2T3MWRFVXLW056972_masterArray_0_3957",
         "2021-04-21-21-45-12_2T3MWRFVXLW056972_masterArray_1_3621",
         "2021-04-22-12-47-13_2T3MWRFVXLW056972_masterArray_1_5292",
         "2021-04-26-21-13-18_2T3MWRFVXLW056972_masterArray_0_4595",
         "2021-04-26-21-13-18_2T3MWRFVXLW056972_masterArray_1_4664",
         "2021-04-27-21-37-32_2T3MWRFVXLW056972_masterArray_0_3836",
         "2021-04-27-21-37-32_2T3MWRFVXLW056972_masterArray_1_3558",
         "2021-04-29-21-16-14_2T3MWRFVXLW056972_masterArray_0_4190",
         "2021-04-29-21-16-14_2T3MWRFVXLW056972_masterArray_1_4005"]

import time


for n in range(1, 5):
    for i in range(len(INFILE)):
        t0 = time.time()

        base_path = "/home/aboudy/Documents/Code/trajectory_training/dataset/data_v2_preprocessed_west"
        downstream_path = os.path.join(base_path, OUTFILE[i])

        base_path2 = "/home/aboudy/Documents/Code/DDPGfD/ddpgfd/scripts/demo/random/emissions"
        infile = os.path.join(base_path2, INFILE[i], "emissions/emissions_1.csv")

        data = main(infile, downstream_path, n)

        with open("random/th/{}/n={}/{}.pkl".format(H_HIGH, n, OUTFILE[i]), "wb") as f:
            pickle.dump(data, f)

        print(n, i, time.time() - t0)

AttributeError: 'DataFrame' object has no attribute 'pos'

### MPC

In [10]:
INFILE=list(range(17))

OUTFILE=sorted([
    '2021-04-16-12-34-41_2T3MWRFVXLW056972_masterArray_0_5778',
#     '2021-03-24-12-39-15_2T3MWRFVXLW056972_masterArray_0_6438',
    '2021-03-08-22-35-14_2T3MWRFVXLW056972_masterArray_1_4927',
    '2021-03-24-21-34-31_2T3MWRFVXLW056972_masterArray_0_4937',
    '2021-04-22-12-47-13_2T3MWRFVXLW056972_masterArray_1_5292',
    '2021-03-09-13-35-04_2T3MWRFVXLW056972_masterArray_0_6825',
    '2021-03-15-12-46-38_2T3MWRFVXLW056972_masterArray_1_11342',
    '2021-04-19-12-27-33_2T3MWRFVXLW056972_masterArray_1_6483',
    '2021-03-09-13-35-04_2T3MWRFVXLW056972_masterArray_1_4938',
    '2021-03-17-21-37-10_2T3MWRFVXLW056972_masterArray_0_4463',
#     '2021-04-22-12-47-13_2T3MWRFVXLW056972_masterArray_0_7050',
    '2021-04-12-21-34-57_2T3MWRFVXLW056972_masterArray_0_4796',
    '2021-04-07-12-33-03_2T3MWRFVXLW056972_masterArray_0_11294',
    '2021-04-19-12-27-33_2T3MWRFVXLW056972_masterArray_0_16467',
    '2021-04-07-12-33-03_2T3MWRFVXLW056972_masterArray_1_6116',
    '2021-03-29-12-47-15_2T3MWRFVXLW056972_masterArray_0_5016',
    '2021-03-12-22-20-57_2T3MWRFVXLW056972_masterArray_0_5672',
    '2021-04-27-21-37-32_2T3MWRFVXLW056972_masterArray_0_3836',
    '2021-03-15-12-46-38_2T3MWRFVXLW056972_masterArray_0_4917',
])

import time

print(len(OUTFILE))

for n in range(1, 5):
    for i in range(len(INFILE)):
        t0 = time.time()

        base_path = "/home/aboudy/Documents/Code/trajectory_training/dataset/data_v2_preprocessed_west"
        downstream_path = os.path.join(base_path, OUTFILE[i])

        infile = "mpc/emissions/{}.csv".format(INFILE[i])

        data = main(infile, downstream_path, n)

        with open("mpc/th+accel/{}/n={}/{}.pkl".format(H_HIGH, n, OUTFILE[i]), "wb") as f:
            pickle.dump(data, f)

        print(n, i, time.time() - t0)

17
1 0 0.17846083641052246
1 1 0.23133087158203125
1 2 0.16844582557678223
1 3 0.19582223892211914
1 4 0.16726303100585938
1 5 0.3958277702331543
1 6 0.15259408950805664
1 7 0.16965079307556152
1 8 0.20142745971679688
1 9 0.48784446716308594
1 10 0.2203083038330078
1 11 0.16363215446472168
1 12 0.1966075897216797
1 13 0.5725865364074707
1 14 0.22064423561096191
1 15 0.18192601203918457
1 16 0.1321425437927246
2 0 0.16651010513305664
2 1 0.23346924781799316
2 2 0.16765856742858887
2 3 0.19447708129882812
2 4 0.16724348068237305
2 5 0.420046329498291
2 6 0.16983461380004883
2 7 0.17043352127075195
2 8 0.1718120574951172
2 9 0.4321887493133545
2 10 0.2074570655822754
2 11 0.16536617279052734
2 12 0.19707322120666504
2 13 0.5700068473815918
2 14 0.22110247611999512
2 15 0.18203520774841309
2 16 0.13015151023864746
3 0 0.16753911972045898
3 1 0.23716521263122559
3 2 0.17437982559204102
3 3 0.19582390785217285
3 4 0.16871929168701172
3 5 0.3964803218841553
3 6 0.15440869331359863
3 7 0.17178